# Train models

> Tutorial 1: Train a Flood Extent segmentation model using the WorldFloods dataset

## Step 0: Notebook setup
    - Configure notebook basics
    - Configure GCP Credentials

In [5]:
import sys, os
from pathlib import Path

## Step 1: Setup Configuration file
    - Load configuration file from local device or gcs

In [1]:
from ml4floods.models.config_setup import get_default_config
import pkg_resources

# Set filepath to configuration files
# config_fp = 'path/to/worldfloods_template.json'
config_fp = pkg_resources.resource_filename("ml4floods","models/configurations/worldfloods_template.json")

config = get_default_config(config_fp)

## Step 1.a: Seed everything for reproducibility

In [2]:
from pytorch_lightning import seed_everything
# Seed
seed_everything(config.seed)

Global seed set to 12


12

## Step 1.b: Make it a unique experiment
    - 'experiment_name' is used to specify the folder in which to save models and associated files

In [3]:
config.experiment_name = 'training_demo'

## Step 2: Setup Dataloader
    - 'loader_type' can be one of 'local' which assumes the images are already saved locally, or 'bucket' which will load images directly from the bucket specified in 'bucket_id'. If set to local and the dataset is not found in the path `config.data_params.path_to_splits` it will trigger the download of the data

In [5]:
from ml4floods.models.dataset_setup import get_dataset

config.data_params.loader_type = 'local'
config.data_params.path_to_splits = "/worldfloods/public/"
dataset = get_dataset(config.data_params)

Using local dataset for this run
train 196648  tiles
val 1284  tiles
test 11  tiles


## Step 3: Setup Model
     - 'train' = True specifies that we are training a new model from scratch
     - get_model(args) constructs a pytorch lightning model using the configuration specified in 'config.model_params'

In [6]:
 # folder to store the trained model (it will create a subfolder with the name of the experiment)
config.model_params.model_folder = "models" 
os.makedirs("models", exist_ok=True)

config.model_params

{'model_folder': 'models',
 'model_version': 'v1',
 'hyperparameters': {'max_tile_size': 256,
  'metric_monitor': 'val_dice_loss',
  'channel_configuration': 'all',
  'label_names': ['land', 'water', 'cloud'],
  'weight_per_class': [1.93445299, 36.60054169, 2.19400729],
  'model_type': 'linear',
  'num_classes': 3,
  'max_epochs': 10,
  'val_every': 1,
  'lr': 0.0001,
  'lr_decay': 0.5,
  'lr_patience': 2,
  'num_channels': 13},
 'train': True,
 'test': True}

In [7]:
from ml4floods.models.model_setup import get_model

config.model_params.test = False
config.model_params.train = True
model = get_model(config.model_params)
model

WorldFloodsModel(
  (network): SimpleLinear(
    (conv): Sequential(
      (0): Conv2d(13, 3, kernel_size=(1, 1), stride=(1, 1))
    )
  )
)

## Step 4: Set up Weights and Biases Logger for experiment
    - We pass this to the model trainer in a later cell to automaticall log relevant metrics to wandb

In [8]:
import wandb
from pytorch_lightning.loggers import WandbLogger

# UNCOMMENT ON FIRST RUN TO LOGIN TO Weights and Biases (only needs to be done once)
# wandb.login()
# run = wandb.init()

# Specifies who is logging the experiment to wandb
config['wandb_entity'] = 'ml4floods'
# Specifies which wandb project to log to, multiple runs can exist in the same project
config['wandb_project'] = 'worldfloods-notebook-demo-project'

wandb_logger = WandbLogger(
    name=config.experiment_name,
    project=config.wandb_project, 
    entity=config.wandb_entity
)

### Step 5: Setup Lightning Callbacks
    - We implement checkpointing using the ModelCheckpoint callback to save the best performing checkpoints to local/gcs storage
    - We implement early stopping using the EarlyStopping callback to stop training early if there is no performance improvement after 10 epochs from the latest best checkpoint

In [9]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
experiment_path = f"{config.model_params.model_folder}/{config.experiment_name}"

checkpoint_callback = ModelCheckpoint(
    dirpath=f"{experiment_path}/checkpoint",
    save_top_k=True,
    verbose=True,
    monitor='val_dice_loss',
    mode='min',
    prefix=''
)

early_stop_callback = EarlyStopping(
    monitor='val_dice_loss',
    patience=10,
    strict=False,
    verbose=False,
    mode='min'
)

callbacks = [checkpoint_callback, early_stop_callback]

print(f"{config.model_params.model_folder}/{config.experiment_name}")

models/training_demo


## Step 6: Setup Lighting Trainer
    -- Pytorch Lightning Trainer handles all the rest of the model training for us!
    -- add flags from 
    https://pytorch-lightning.readthedocs.io/en/0.7.5/trainer.html 

In [11]:
from pytorch_lightning import Trainer

config.gpus = '0'  # which gpu to use

config.model_params.hyperparameters.max_epochs = 4 # train for maximum 4 epochs

trainer = Trainer(
    fast_dev_run=False,
    logger=wandb_logger,
    callbacks=callbacks,
    default_root_dir=f"{config.model_params.model_folder}/{config.experiment_name}",
    accumulate_grad_batches=1,
    gradient_clip_val=0.0,
    auto_lr_find=False,
    benchmark=False,
    distributed_backend=None,
    gpus=config.gpus,
    max_epochs=config.model_params.hyperparameters.max_epochs,
    check_val_every_n_epoch=config.model_params.hyperparameters.val_every,
    log_gpu_memory=None,
    resume_from_checkpoint=None
)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


### Start Training!

In [11]:
trainer.fit(model, dataset)


  | Name    | Type         | Params
-----------------------------------------
0 | network | SimpleLinear | 42    
-----------------------------------------
42        Trainable params
0         Non-trainable params
42        Total params
0.000     Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/miniconda3/envs/ml4floods/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Epoch 0:  99%|█████████▉| 6147/6187 [46:53<00:18,  2.18it/s, loss=1.16, v_num=wene]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|█████████▉| 6159/6187 [47:09<00:12,  2.18it/s, loss=1.16, v_num=wene]

/home/gonzalo/ml4floods/src/models/utils/metrics.py:139: RuntimeWarning: divide by zero encountered in true_divide
  recall = true_positive / (true_positive + false_negative)


...

Epoch 2, global step 18437: val_dice_loss reached 0.67237 (best 0.67237), saving model to "gs://ml4cc_data_lake/0_DEV/2_Mart/2_MLModelMart/worldfloods-notebook-training-demo/checkpoint/epoch=2-step=18437.ckpt" as top True


Epoch 3:  99%|█████████▉| 6146/6187 [48:58<00:19,  2.09it/s, loss=0.987, v_num=wene]
Validating: 0it [00:00, ?it/s]
Epoch 3: 100%|██████████| 6187/6187 [49:52<00:00,  2.07it/s, loss=0.987, v_num=wene]
                                                           

Epoch 3, global step 24583: val_dice_loss reached 0.66332 (best 0.66332), saving model to "gs://ml4cc_data_lake/0_DEV/2_Mart/2_MLModelMart/worldfloods-notebook-training-demo/checkpoint/epoch=3-step=24583.ckpt" as top True


Epoch 3: 100%|██████████| 6187/6187 [49:54<00:00,  2.07it/s, loss=0.987, v_num=wene]


1

## Step 7: Save trained model
    - Save model to local/gcs along with configuration file used to conduct training!

In [13]:
import torch
from pytorch_lightning.utilities.cloud_io import atomic_save
from ml4floods.models.config_setup import save_json

# Save in the cloud and in the wandb logger save dir
atomic_save(model.state_dict(), f"{experiment_path}/model.pt")
torch.save(model.state_dict(), os.path.join(wandb_logger.save_dir, 'model.pt'))
wandb.save(os.path.join(wandb_logger.save_dir, 'model.pt')) # Copy weights to weights and biases server
wandb.finish()

# Save cofig file in experiment_path
config_file_path = f"{experiment_path}/config.json"
save_json(config, config_file_path)

wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")
Run pip install nbformat to save notebook history


_runtime,25660
_timestamp,1616105878
_step,24583
loss,0.87475
epoch,3
val_bce_loss,1.81653
val_dice_loss,0.66332
val_recall land,0.86352
val_recall water,inf
val_recall cloud,inf
val_iou land,0.78074


_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▇▆▆▄▄▆▅▄▃▃▅▃▂▂▃▃▃▃▄▅▇▂▂▃▃▁▄▅▂▁▂▂▂▃▆▂▂▂▂
epoch,▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▆▆▆▆▆▆▆▆▆▆██████████
val_bce_loss,█▄▂▁
val_dice_loss,█▄▂▁
val_recall land,▁▆██
val_recall water,
val_recall cloud,
val_iou land,▁▆██


All Done - Now head to the Model Inference Tutorial to see how your model performed!